In [1]:
import pandas as pd
import json

import os
os.makedirs('data', exist_ok=True)

In [2]:
data = pd.read_csv('imdb_dataset.csv')

In [3]:
for key in ['train', 'dev', 'test'] :
    data_key = data[data.exp_split == key].to_dict(orient='records')
    data_key = [{'document' : x['text'], 'label' : x['label']} for x in data_key]
    f = open('data/' + key + '.jsonl', 'w')
    f.write('\n'.join([json.dumps(line) for line in data_key]))
    f.close()